# tf.kerasのカスタムloss/metricsデモ

## サンプルデータセット用意

In [1]:
import numpy as np
import tensorflow as tf


In [2]:
with tf.device("/CPU:0"):
    MAX_GROUP_ID_NUMBER = 10
    SAMPLE_SIZE = 1000
    target = tf.random.uniform([SAMPLE_SIZE], maxval=2, dtype=tf.int32)
    feat_grpby = tf.random.uniform(
        [SAMPLE_SIZE], minval=1, maxval=MAX_GROUP_ID_NUMBER + 1, dtype=tf.int32
    )
    feat_grpby_ohe = tf.one_hot(feat_grpby, depth=tf.reduce_max(feat_grpby))
    feat_grouped = tf.random.normal([SAMPLE_SIZE, 2])

    explains = tf.concat([feat_grouped, feat_grpby_ohe], axis=1)
    target_w_feat_grpby = tf.stack([target, feat_grpby], axis=1)


2022-01-07 03:21:20.412321: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-01-07 03:21:20.477606: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-01-07 03:21:20.478037: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-01-07 03:21:20.479778: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate

In [3]:
from random import random

group_weight = {k: 30.0 * random() for k in range(1, MAX_GROUP_ID_NUMBER + 1)}
group_weight


{1: 27.42537617089975,
 2: 3.4833571698002985,
 3: 0.6176417763599207,
 4: 17.3279240541167,
 5: 10.436575357373837,
 6: 12.52318986521875,
 7: 23.542190564539062,
 8: 22.196339393202983,
 9: 20.412816578333846,
 10: 17.838767807901256}

## Custom Loss

NOTE:

微分可能な処理でなければならない．
勾配情報を保持できるかは以下を参照のこと

- https://www.tensorflow.org/api_docs/python/tf/raw_ops/
- https://stackoverflow.com/a/44575034


In [4]:
from typing import Optional

from tensorflow.keras.losses import Loss, binary_crossentropy as bce
import pandas as pd


class GroupWeightedBinaryCrossentropy(Loss):
    def __init__(self, group_weight: dict, name="GroupWeightedBCE"):
        super().__init__(name=name)
        self.name = name

        if not isinstance(group_weight, dict):
            errmsg = "For the feature column to be grouped, "
            errmsg += "give the weights of each ID as arguments in dict format."
            raise TypeError(errmsg)
        self.group_weight = group_weight

        self.avg_loss_grpby = {
            k: tf.convert_to_tensor(0, dtype=tf.float32) for k in group_weight.keys()
        }
        self.global_avg_loss = tf.convert_to_tensor(0, dtype=tf.float32)
        self.batch_cnt = tf.convert_to_tensor(0, dtype=tf.float32) 

    def _update_loss_for_each_group(self, y_true_w_id, y_pred):
        y_true = y_true_w_id[:, 0]
        feat_id = y_true_w_id[:, 1]

        y_pred = tf.convert_to_tensor(y_pred)
        y_pred = tf.cast(y_pred, tf.float32)
        y_pred = tf.reshape(y_pred, shape=y_true.shape)
        y_true = tf.cast(y_true, y_pred.dtype)

        # Group内，IDごとの平均lossを計算
        for k in self.group_weight.keys():
            selected_indices = tf.where(tf.equal(feat_id, k))
            # NaN対策（IDが存在しない場合）
            if selected_indices.shape[0] < 1:
                continue
            y_pred_grpby = tf.gather_nd(y_pred, selected_indices)
            y_true_grpby = tf.gather_nd(y_true, selected_indices)
            y_pred_grpby = tf.reshape(y_pred_grpby, shape=[1, -1])
            y_true_grpby = tf.reshape(y_true_grpby, shape=[1, -1])
            loss_for_group = bce(y_true_grpby, y_pred_grpby)[0]

            multiplied = tf.multiply(self.avg_loss_grpby[k], self.batch_cnt)
            numerator = tf.add(multiplied, loss_for_group)
            denominator = tf.add(self.batch_cnt, 1)
            self.avg_loss_grpby[k] = tf.divide(numerator, denominator)

        self.batch_cnt = tf.add(self.batch_cnt, 1)

    def _update_global_loss(self, avg_loss_grpby, group_weight: dict):
        sum_loss_weighted = tf.convert_to_tensor(0, dtype=tf.float32)
        denominator = tf.convert_to_tensor(0, dtype=tf.float32)
        for k, avg_loss in avg_loss_grpby.items():
            weight = group_weight[k]
            multiplied = tf.multiply(avg_loss, weight)
            sum_loss_weighted = tf.add(sum_loss_weighted, multiplied)
            denominator = tf.add(denominator, weight)

        self.global_avg_loss = tf.divide(sum_loss_weighted, denominator)

    def call(self, y_true_w_id, y_pred):
        # model.compile, fit時の内部挙動でエラーが吐きそうなため
        # y_trueはdictではなくarray likeに与える
        self._update_loss_for_each_group(y_true_w_id, y_pred)
        self._update_global_loss(self.avg_loss_grpby, self.group_weight)

        return self.global_avg_loss


In [5]:
# 計算値確認
tmp = GroupWeightedBinaryCrossentropy(group_weight=group_weight)
y_pred = tf.random.normal(shape=[target_w_feat_grpby.shape[0]])
global_weighted_loss = tmp.call(target_w_feat_grpby, y_pred)
global_weighted_loss


<tf.Tensor: shape=(), dtype=float32, numpy=5.503593>

In [6]:
# 重み付け確認
wo_weight = np.mean([v.numpy() for v in tmp.avg_loss_grpby.values()])

assert wo_weight != global_weighted_loss


#### NG1

In [7]:
"""
from typing import Optional

from tensorflow.keras.losses import Loss
import pandas as pd


class GroupWeightedDeviation(Loss):
    def __init__(self, group_weight: dict, name="Deviation"):
        super().__init__(name=name)
        self.name = name
        if isinstance(group_weight, dict):
            self.group_weight = pd.Series(group_weight, name="GroupWeight")
            self.group_weight.sort_index(ascending=True, inplace=True)
        else:
            errmsg = "For the feature column to be grouped, "
            errmsg += "give the weights of each ID as arguments in dict format."
            raise TypeError(errmsg)
        self.global_avg_loss = 0.0
        self.batch_cnt = 0
        self.avg_loss_grpby = pd.Series(0.0, index=group_weight.keys(), name=name)
        self.avg_loss_grpby.sort_index(ascending=True, inplace=True)

    def call(self, y_true_w_id, y_pred, var_deal_zerodiv=0.0001):
        # model.compile, fit時の内部挙動でエラーが吐きそうなため
        # y_trueはdictではなくarray likeに与える
        y_true = y_true_w_id[:, 0]
        feat_id = y_true_w_id[:, 1]

        y_pred = tf.convert_to_tensor(y_pred)
        y_pred = tf.cast(y_pred, tf.float32)
        y_pred = tf.reshape(y_pred, shape=y_true.shape)
        y_true = tf.cast(y_true, y_pred.dtype)

        deviation = tf.math.abs(y_true / (y_pred + var_deal_zerodiv) - 1)
        loss = pd.Series(deviation.numpy(), index=feat_id, name=self.name)
        loss = loss.groupby(by=loss.index).mean()

        # 単純な+で演算した場合，avg_loss_grpbyのIDがlossに含まれていない場合
        # 該当IDの演算和がNaNになるため fill_valを指定したaddを利用
        denom = (self.avg_loss_grpby * self.batch_cnt).add(loss, fill_value=0.0)
        self.avg_loss_grpby = denom / (self.batch_cnt + 1)

        self.global_avg_loss = (
            self.avg_loss_grpby * self.group_weight
        ).sum() / self.group_weight.sum()

        self.batch_cnt += 1

        return tf.convert_to_tensor(self.global_avg_loss, dtype=tf.float32)
"""


'\nfrom typing import Optional\n\nfrom tensorflow.keras.losses import Loss\nimport pandas as pd\n\n\nclass GroupWeightedDeviation(Loss):\n    def __init__(self, group_weight: dict, name="Deviation"):\n        super().__init__(name=name)\n        self.name = name\n        if isinstance(group_weight, dict):\n            self.group_weight = pd.Series(group_weight, name="GroupWeight")\n            self.group_weight.sort_index(ascending=True, inplace=True)\n        else:\n            errmsg = "For the feature column to be grouped, "\n            errmsg += "give the weights of each ID as arguments in dict format."\n            raise TypeError(errmsg)\n        self.global_avg_loss = 0.0\n        self.batch_cnt = 0\n        self.avg_loss_grpby = pd.Series(0.0, index=group_weight.keys(), name=name)\n        self.avg_loss_grpby.sort_index(ascending=True, inplace=True)\n\n    def call(self, y_true_w_id, y_pred, var_deal_zerodiv=0.0001):\n        # model.compile, fit時の内部挙動でエラーが吐きそうなため\n        # y

#### NG2

逆伝搬の勾配情報が保持できなければcustom lossを確保できないので，以下は対応していない

https://www.tensorflow.org/api_docs/python/tf/raw_ops/
で確認できる



In [8]:
"""
from typing import Optional

from tensorflow.keras.losses import Loss, binary_crossentropy as bce
import pandas as pd


class GroupWeightedBinaryCrossentropy(Loss):
    def __init__(self, group_weight: dict, name="Deviation"):
        super().__init__(name=name)
        self.name = name
        if isinstance(group_weight, dict):
            self.group_weight = pd.Series(
                group_weight, name="GroupWeight", dtype=np.float32
            )
            self.group_weight.sort_index(ascending=True, inplace=True)
        else:
            errmsg = "For the feature column to be grouped, "
            errmsg += "give the weights of each ID as arguments in dict format."
            raise TypeError(errmsg)
        self.global_avg_loss = 0.0
        self.batch_cnt = 0
        self.avg_loss_grpby = pd.Series(0.0, index=group_weight.keys(), name=name)
        self.avg_loss_grpby.sort_index(ascending=True, inplace=True)

    def call(self, y_true_w_id, y_pred):
        # model.compile, fit時の内部挙動でエラーが吐きそうなため
        # y_trueはdictではなくarray likeに与える
        y_true = y_true_w_id[:, 0]
        feat_id = y_true_w_id[:, 1]

        y_pred = tf.convert_to_tensor(y_pred)
        y_pred = tf.cast(y_pred, tf.float32)
        y_pred = tf.reshape(y_pred, shape=y_true.shape)
        y_true = tf.cast(y_true, y_pred.dtype)

        loss = pd.DataFrame({"y_true": y_true, "y_pred": y_pred}, index=feat_id)
        loss = loss.groupby(by=loss.index).apply(lambda d: bce(d.y_true, d.y_pred))

        # 単純な+で演算した場合，avg_loss_grpbyのIDがlossに含まれていない場合
        # 該当IDの演算和がNaNになるため fill_valを指定したaddを利用
        denom = (self.avg_loss_grpby * self.batch_cnt).add(loss, fill_value=tf.constant(0.0, dtype=tf.float32, shape=[0]))
        #display(denom)
        self.avg_loss_grpby = denom / (self.batch_cnt + 1)

        denom = tf.reduce_sum(self.avg_loss_grpby * self.group_weight)
        self.global_avg_loss = tf.divide(denom, self.group_weight.sum())

        self.batch_cnt += 1

        return self.global_avg_loss
"""


'\nfrom typing import Optional\n\nfrom tensorflow.keras.losses import Loss, binary_crossentropy as bce\nimport pandas as pd\n\n\nclass GroupWeightedBinaryCrossentropy(Loss):\n    def __init__(self, group_weight: dict, name="Deviation"):\n        super().__init__(name=name)\n        self.name = name\n        if isinstance(group_weight, dict):\n            self.group_weight = pd.Series(\n                group_weight, name="GroupWeight", dtype=np.float32\n            )\n            self.group_weight.sort_index(ascending=True, inplace=True)\n        else:\n            errmsg = "For the feature column to be grouped, "\n            errmsg += "give the weights of each ID as arguments in dict format."\n            raise TypeError(errmsg)\n        self.global_avg_loss = 0.0\n        self.batch_cnt = 0\n        self.avg_loss_grpby = pd.Series(0.0, index=group_weight.keys(), name=name)\n        self.avg_loss_grpby.sort_index(ascending=True, inplace=True)\n\n    def call(self, y_true_w_id, y_pred)

## 適当なモデルを用意

In [9]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.dense1 = tf.keras.layers.Dense(4, activation=tf.nn.relu)
        self.dense2 = tf.keras.layers.Dense(1, activation=tf.nn.softmax)

    def call(self, inputs):
        x = self.dense1(inputs)
        return self.dense2(x)


In [10]:
model = MyModel()
# TODO: 適当にTrainする
model.compile(
    loss=GroupWeightedBinaryCrossentropy(group_weight=group_weight),
    optimizer="adam",
    run_eagerly=True,
)


## 学習

### Train-test split

In [11]:
from sklearn.model_selection import train_test_split


train_indices, valid_indices = train_test_split(np.arange(SAMPLE_SIZE), train_size=0.9)

with tf.device("/CPU:0"):
    X_train = tf.gather(explains, train_indices, axis=0)
    X_valid = tf.gather(explains, valid_indices, axis=0)
    y_train_w_feat_grpby = tf.gather(target_w_feat_grpby, train_indices, axis=0)
    y_valid_w_feat_grpby = tf.gather(target_w_feat_grpby, valid_indices, axis=0)


## Fit

In [12]:
with tf.device("/CPU:0"):
    model.fit(
        X_train,
        y_train_w_feat_grpby,
        validation_data=(X_valid, y_valid_w_feat_grpby),
        epochs=1,
        batch_size=32,
    )


 1/29 [>.............................] - ETA: 3s - loss: 9.1056

2022-01-07 03:21:23.555740: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


29/29 [==============================] - 1s 29ms/step - loss: 7.9877 - val_loss: 7.5598
